#### 1. 기간 설정시 날짜 입력 안됨 문제

In [ ]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException, TimeoutException

In [ ]:
start_date = '2018.01.01'
end_date   = '2018.12.31'

In [ ]:
driver = webdriver.Chrome('chromedriver')
url = "https://search.naver.com/search.naver?where=news&query=&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={}&de={}&docid=&nso=so%3Ar%2Cp%3Afrom{}to{}%2Ca%3Aall&mynews=0&refresh_start=0&related=0".format(start_date, end_date, start_date.replace(".",""), end_date.replace(".",""))
driver.get(url)

In [ ]:
data_dic = {
     'title' : []
    ,'content' : []
    ,'date' : []
    ,'url' : []
}

In [ ]:
# 검색옵션 클릭
# driver.find_element_by_css_selector('#_search_option_btn').click()

# 최신순 선택
driver.find_element_by_css_selector('#snb > div > ul > li:nth-child(3) > a').click()
driver.find_element_by_css_selector('#snb > div > ul > li.menu.selected > div > ul > li:nth-child(2) > a').click()

# 언론사 선택
driver.find_element_by_css_selector('#news_popup > a').click()
# 연합뉴스 체크
driver.find_element_by_css_selector('#ca_1001').click()
# 확인
driver.find_element_by_css_selector('#_nx_option_media > div.con_bx > div.view_btn > button.impact._submit_btn').click()

In [ ]:
# 뉴스 키워드 입력
keyword = '날씨'
# 검색
driver.find_element_by_css_selector('#news_inner_search_query').send_keys(keyword)
driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div > div.news_srch > span > button').click()
# 최신순 정렬
driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div.section_head > div.news_option > ul > li.select').click()

In [ ]:
# 전체 번호 세팅
total_page = driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div.section_head > div.title_desc.all_my > span').text
total_page = total_page.split('/')
total_page = total_page[1]
total_page = total_page.replace(" ", "")
total_page = total_page.replace(",", "")
total_page = total_page.replace("건", "")
total_page = int(total_page)

# 현재 번호
current_page = driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div.paging > strong').text
current_page = int(current_page)

In [ ]:
# 다 끝났으면 다음 페이지로 이동하는 모듈
def moveNextPage():
    global current_page
    current_page = current_page + 1
    # 페이징 태그 가져오기
    paging_elements = driver.find_elements_by_css_selector(".paging > a")
    for paging_num in paging_elements:
        if(paging_num.text == str(current_page)):
            paging_num.click()
            break

In [ ]:
def getContent():
    content = ''
    date = ''
    url = ''
    try:
        content = driver.find_element_by_css_selector('#articleBodyContents').text
        date = "".join(re.findall('\d+',driver.find_element_by_css_selector('#main_content > div.article_header > div.article_info > div > span:nth-child(1)').text[0:10]))
        url = driver.current_url
    except:
        print(driver.current_url,"오류 발생")
        content = "오류"
    finally:
        return content, date, url

In [ ]:
while(True):
    page_content = driver.find_elements_by_css_selector('.type01 > li')
    for content in page_content:
        # 제목 담기
        data_dic['title'].append(content.find_element_by_css_selector('._sp_each_title').text)
        
        # 링크 클릭(네이버 뉴스로 대체)
        content.find_element_by_css_selector('._sp_each_url').click()
        # 최근 열린 탭으로 전환
        driver.switch_to.window(driver.window_handles[-1])
        
        ### 새로운 탭에서 작업 ###
        [content, date, url] = getContent()
        data_dic['content'].append(content)
        data_dic['date'].append(date)
        data_dic['url'].append(url)
        
        # 현재 탭 닫기
        driver.close()
        # 맨 처음 탭으로 변경(0번 탭)
        driver.switch_to.window(driver.window_handles[0])
    if(current_page * 10 >= total_page):
        print(current_page*10,'크롤링 끝!')
        break
    else:
        moveNextPage()

In [ ]:
df = pd.DataFrame.from_dict(data_dic)
# df.to_csv('20152018날씨.csv', index=False, encoding='utf-8-sig')
df.to_csv('test.csv', index=False, encoding='utf-8-sig')